reading timestamps to filter out ones we are interested in

In [1]:
import pandas as pd

timestamps_data = pd.read_csv("data_csv/timestamps_ext.csv")
timestamps_data.drop(columns=[f"p{i}_championId" for i in range(1, 11)], inplace=True)
target = pd.read_csv("data_csv/first_win.csv")
target.columns = ["matchId", "first_win"]
data = timestamps_data.merge(target, on="matchId")
del target

with open("data_csv/timestamps_ext_columns.txt", "w") as f:
    f.write( "\n".join(timestamps_data.columns))
data_cleaned = data.dropna()
del timestamps_data

In [2]:
timestamps_starting = data_cleaned[data_cleaned["timeStamp"] == 0]
timestamps_20p = data_cleaned.loc[
    data_cleaned.groupby("matchId")["percentTimeStamp"].apply(
        lambda x: (x - 0.2).abs().idxmin()
    )
]
timestamps_40p = data_cleaned.loc[
    data_cleaned.groupby("matchId")["percentTimeStamp"].apply(
        lambda x: (x - 0.4).abs().idxmin()
    )
]
timestamps_60p = data_cleaned.loc[
    data_cleaned.groupby("matchId")["percentTimeStamp"].apply(
        lambda x: (x - 0.6).abs().idxmin()
    )
]
timestamps_80p = data_cleaned.loc[
    data_cleaned.groupby("matchId")["percentTimeStamp"].apply(
        lambda x: (x - 0.8).abs().idxmin()
    )
]
timestamps_100p = data_cleaned.loc[
    data_cleaned.groupby("matchId")["percentTimeStamp"].apply(
        lambda x: (x - 1.0).abs().idxmin()
    )
]


In [3]:
#make data_cleaned only include whats in timestamps_starting .. 100

data_cleaned = pd.concat(
    [timestamps_starting, timestamps_20p, timestamps_40p, timestamps_60p, timestamps_80p, timestamps_100p]
)

loading matches data to filtered timestamps

In [4]:
matches_data = pd.read_csv("data_csv/matches.csv")
with open("data_csv/match_columns.txt","w") as f:
    f.write("\n".join(matches_data.columns))
    
columns = [] 
columns.extend([f"p{i}_teamPosition" for i in range(1, 11)])
columns.extend([f"p{i}_summonerLevel" for i in range(1, 11)])
columns.extend([f"p{i}_summoner1Id" for i in range(1, 11)])
columns.extend([f"p{i}_summoner2Id" for i in range(1, 11)])
columns.extend([f"p{i}_role" for i in range(1, 11)])
columns.extend([f"p{i}_lane" for i in range(1, 11)])
columns.extend([f"p{i}_individualPosition" for i in range(1, 11)])
columns.extend([f"p{i}_championId" for i in range(1, 11)])
columns.extend(["matchId","gameId","gameVersion", "gameCreation","gameStartTimestamp","gameEndTimestamp", "gameDuration" ])


matches_data = matches_data[columns]
for c in columns:
    print(c)
    print(matches_data[c].head())
    print("\n")

print(len(matches_data.columns))
data_cleaned = data_cleaned.merge(matches_data, on="matchId")
print(len(data_cleaned.columns))
#del matches_data
#call garbagecollector 
import gc
gc.collect()

p1_teamPosition
0    TOP
1    TOP
2    TOP
3    TOP
4    TOP
Name: p1_teamPosition, dtype: object


p2_teamPosition
0    JUNGLE
1    JUNGLE
2    JUNGLE
3    JUNGLE
4    JUNGLE
Name: p2_teamPosition, dtype: object


p3_teamPosition
0    MIDDLE
1    MIDDLE
2    MIDDLE
3    MIDDLE
4    MIDDLE
Name: p3_teamPosition, dtype: object


p4_teamPosition
0    BOTTOM
1    BOTTOM
2    BOTTOM
3    BOTTOM
4    BOTTOM
Name: p4_teamPosition, dtype: object


p5_teamPosition
0    UTILITY
1    UTILITY
2    UTILITY
3    UTILITY
4    UTILITY
Name: p5_teamPosition, dtype: object


p6_teamPosition
0    TOP
1    TOP
2    TOP
3    TOP
4    TOP
Name: p6_teamPosition, dtype: object


p7_teamPosition
0    JUNGLE
1    JUNGLE
2    JUNGLE
3    JUNGLE
4    JUNGLE
Name: p7_teamPosition, dtype: object


p8_teamPosition
0    MIDDLE
1    MIDDLE
2    MIDDLE
3    MIDDLE
4    MIDDLE
Name: p8_teamPosition, dtype: object


p9_teamPosition
0    BOTTOM
1    BOTTOM
2    BOTTOM
3    BOTTOM
4    BOTTOM
Name: p9_teamPosition, dtype:

0

In [5]:
print(len(data_cleaned.columns))

551


if matches.csv is having player ids in columns 0-9 instead of 1-10 uncomment below and paste as first line whats in output into matches.csv

In [6]:
columns_to_drop = ["gameMode", "gameName", "gameType", "gameVersion", "mapId" ]
data_cleaned.drop(columns=columns_to_drop, inplace=True, errors="ignore")
data_cleaned.drop(columns= ["p{}_summonerId".format(i) for i in range(1, 11)], inplace=True, errors="ignore")
data_cleaned.drop(columns= ["p{}_championName".format(i) for i in range(1, 11)], inplace=True, errors="ignore")

columns_to_drop = ["p{}_win".format(i) for i in range(1, 11)]
data_cleaned.drop(columns=["p{}_win".format(i) for i in range(1,11)], inplace=True, errors="ignore")
data_cleaned.drop(columns=["p{}_nexusLost".format(i) for i in range(1,11)], inplace=True, errors="ignore")
data_cleaned.drop(columns=["p{}_nexusKills".format(i) for i in range(1,11)], inplace=True, errors="ignore")

data_cleaned.drop(columns= ["p{}_summonerName".format(i) for i in range(1, 11)], inplace=True, errors="ignore")
data_cleaned.drop(columns= ["p{}_riotIdTagline".format(i) for i in range(1, 11)], inplace=True, errors="ignore")
data_cleaned.drop(columns= ["tournamentCode"], inplace=True, errors="ignore")
data_cleaned.drop(columns= ["p{}_puuid".format(i) for i in range(1, 11)], inplace=True, errors="ignore")

In [7]:
len(data_cleaned.columns)

550

In [8]:
#reset index
data_cleaned.reset_index(drop=True, inplace=True)
print(data_cleaned.shape)

(145302, 550)


In [9]:
del timestamps_100p
del timestamps_80p
del timestamps_60p
del timestamps_40p
del timestamps_20p
del timestamps_starting
gc.collect()

0

In [10]:
#show how many Na values are in each column and order them by the number of Na values
na_values = data_cleaned.isna().sum().sort_values(ascending=False)
print(na_values)

p8_teamPosition                          18
p5_teamPosition                          12
p1_teamPosition                           6
matchId                                   0
p6_damageStats_magicDamageTaken           0
                                         ..
p6_championStats_bonusMagicPenPercent     0
p5_championStats_bonusMagicPenPercent     0
p4_championStats_bonusMagicPenPercent     0
p3_championStats_bonusMagicPenPercent     0
gameDuration_y                            0
Length: 550, dtype: int64


In [11]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


def evaluate_model(model, X, y):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42
    )

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)

    print(f"   Model: {model.__class__.__name__}")
    print(f"Accuracy: {accuracy:.4f}\n")


models = [
    LogisticRegression(random_state=42, n_jobs=-1),
    DecisionTreeClassifier(random_state=42),
    RandomForestClassifier(random_state=42, n_jobs=-1),
    HistGradientBoostingClassifier(random_state=42),
]

In [12]:
from sklearn.model_selection import cross_val_score


def cross_val_model(model, X, y):
    # Perform cross-validation and get accuracy scores
    scores = cross_val_score(model, X, y, cv=5, n_jobs=-1, scoring="accuracy")
    # Print the model's class name
    print(f"Model: {model.__class__.__name__}")
    # Print all cross-validation scores
    print(f"Cross-validation scores (Accuracy): {scores}")
    # Print the mean cross-validation score
    print(f"Mean cross-validation score (Accuracy): {scores.mean():.4f}\n")

In [13]:
def test_timestamp(timestamp):
    X, y, name = timestamp
    print("Timestamp at {} percent\n".format(name))
    for model in models:
        evaluate_model(model, X, y)

    # uncomment to also test with cross_validation - takes more time and results are similar so I left it commented out for now
    # for model in models:
    #     cross_val_model(model, X, y)

In [14]:
import regex as re
others = ['matchId', 'timeStamp', 'percentTimeStamp', "gameDuration"]

blue_team_pattern = re.compile(r'p[1-5]_(.*)')
red_team_pattern = re.compile(r'p(6|7|8|9|10)_(.*)')

blue_team_columns = {}
red_team_columns = {}


cols_to_onehot_encode = []
cols_to_onehot_encode.extend([f"p{i}_teamPosition" for i in range(1, 11)])
cols_to_onehot_encode.extend([f"p{i}_role" for i in range(1, 11)])
cols_to_onehot_encode.extend([f"p{i}_lane" for i in range(1, 11)])
cols_to_onehot_encode.extend([f"p{i}_individualPosition" for i in range(1, 11)])
cols_to_onehot_encode.extend([f"p{i}_summoner1Id" for i in range(1, 11)])
cols_to_onehot_encode.extend([f"p{i}_summoner2Id" for i in range(1, 11)])
cols_to_onehot_encode.extend([f"p{i}_championId" for i in range(1, 11)])


for col in data_cleaned.columns:
    if col not in cols_to_onehot_encode:
        if blue_team_pattern.match(col):
            feature_name = blue_team_pattern.match(col).group(1)
            if feature_name not in blue_team_columns:
                blue_team_columns[feature_name] = []
            blue_team_columns[feature_name].append(col)
        elif red_team_pattern.match(col):
            feature_name = red_team_pattern.match(col).group(2)
            if feature_name not in red_team_columns:
                red_team_columns[feature_name] = []
            red_team_columns[feature_name].append(col)

def aggregate_team_columns(data, team_columns, team_prefix):
    for feature, columns in team_columns.items():
        aggregated_column = f'{team_prefix}_{feature}'
        data[aggregated_column] = data[columns].sum(axis=1)
        print(aggregated_column)
    return data

data_cleaned = aggregate_team_columns(data_cleaned, blue_team_columns, 'blue').copy()
data_cleaned = aggregate_team_columns(data_cleaned, red_team_columns, 'red').copy()

blue_currentGold
blue_goldPerSecond
blue_jungleMinionsKilled
blue_level
blue_minionsKilled
blue_participantId
blue_timeEnemySpentControlled
blue_totalGold
blue_xp
blue_championStats_abilityHaste
blue_championStats_abilityPower
blue_championStats_armor
blue_championStats_armorPen
blue_championStats_armorPenPercent
blue_championStats_attackDamage
blue_championStats_attackSpeed
blue_championStats_bonusArmorPenPercent
blue_championStats_bonusMagicPenPercent
blue_championStats_ccReduction
blue_championStats_cooldownReduction
blue_championStats_health
blue_championStats_healthMax
blue_championStats_healthRegen
blue_championStats_lifesteal
blue_championStats_magicPen
blue_championStats_magicPenPercent
blue_championStats_magicResist
blue_championStats_movementSpeed
blue_championStats_omnivamp
blue_championStats_physicalVamp
blue_championStats_power
blue_championStats_powerMax
blue_championStats_powerRegen
blue_championStats_spellVamp
blue_damageStats_magicDamageDone
blue_damageStats_magicDamag

champion_id one-hot encoding

In [15]:

print(data_cleaned["p1_role"].value_counts())
print(data_cleaned["p1_lane"].value_counts())

#make onehotencoding for all columns in cols_to_onehot_encode
data_cleaned = pd.get_dummies(data_cleaned, columns=cols_to_onehot_encode)


p1_role
SOLO       79872
NONE       37416
SUPPORT    13836
DUO        12984
CARRY       1194
Name: count, dtype: int64
p1_lane
TOP       84624
JUNGLE    37416
NONE      13380
MIDDLE     9126
BOTTOM      756
Name: count, dtype: int64


In [16]:
def print_correlation_features(X,Y, target_name = "first_win"):
    correlations = X.corrwith(Y, numeric_only=True).sort_values(ascending=False)
    print(f"NaN correlations = {correlations.isna().sum()}")
    correlations.dropna(inplace=True)
    
    print("\nTop 10 features most positively correlated with {}:".format(target_name))
    print(correlations.head(10))

    print("\nTop 10 features most negatively correlated with {}:".format(target_name))
    #print tail in reverse order
    print(correlations.tail(10)[::-1])    

In [17]:
#find column with p1_role in it
role_columns = [c for c in data_cleaned.columns if "p1_role" in c]
print(role_columns)
print(data_cleaned["p1_role_CARRY"])

['p1_role_CARRY', 'p1_role_DUO', 'p1_role_NONE', 'p1_role_SOLO', 'p1_role_SUPPORT']
0         False
1         False
2         False
3         False
4         False
          ...  
145297    False
145298    False
145299    False
145300    False
145301    False
Name: p1_role_CARRY, Length: 145302, dtype: bool


In [18]:
print_correlation_features(data_cleaned.drop(columns="p1_role_CARRY"), data_cleaned["p1_role_CARRY"], target_name="p1_role_CARRY")
print_correlation_features(data_cleaned.drop(columns="p1_role_DUO"), data_cleaned["p1_role_DUO"], target_name="p1_role_DUO")
print_correlation_features(data_cleaned.drop(columns="p1_role_SOLO"), data_cleaned["p1_role_SOLO"], target_name="p1_role_SOLO")
print_correlation_features(data_cleaned.drop(columns="p1_role_NONE"), data_cleaned["p1_role_NONE"], target_name="p1_role_NONE")

e:\Users\qesterius\Documents\Studia\sztuczna-inteligencja-w-systemach-inf\.venv\lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
e:\Users\qesterius\Documents\Studia\sztuczna-inteligencja-w-systemach-inf\.venv\lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


NaN correlations = 85

Top 10 features most positively correlated with p1_role_CARRY:
p1_individualPosition_BOTTOM    0.450139
p1_teamPosition_BOTTOM          0.442566
p4_individualPosition_TOP       0.414630
p4_teamPosition_TOP             0.407281
p5_lane_TOP                     0.385323
p1_lane_BOTTOM                  0.317669
p4_role_SOLO                    0.177800
p2_lane_TOP                     0.154786
p4_lane_TOP                     0.130134
p1_championId_222               0.126860
dtype: float64

Top 10 features most negatively correlated with p1_role_CARRY:
p1_teamPosition_TOP            -0.185530
p1_individualPosition_TOP      -0.182589
p4_individualPosition_BOTTOM   -0.162907
p4_teamPosition_BOTTOM         -0.161579
p4_role_CARRY                  -0.116892
p1_role_SOLO                   -0.100570
p5_lane_BOTTOM                 -0.078599
p3_individualPosition_MIDDLE   -0.068324
p3_teamPosition_MIDDLE         -0.066749
p1_lane_JUNGLE                 -0.053605
dtype: float64


e:\Users\qesterius\Documents\Studia\sztuczna-inteligencja-w-systemach-inf\.venv\lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
e:\Users\qesterius\Documents\Studia\sztuczna-inteligencja-w-systemach-inf\.venv\lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


NaN correlations = 85

Top 10 features most positively correlated with p1_role_DUO:
p3_role_DUO                     0.597286
p1_lane_MIDDLE                  0.466630
p3_lane_TOP                     0.434780
p4_lane_TOP                     0.174953
p5_role_SOLO                    0.079231
p3_individualPosition_TOP       0.032489
p1_individualPosition_MIDDLE    0.032329
p1_teamPosition_MIDDLE          0.032208
p3_teamPosition_TOP             0.031968
p4_role_SOLO                    0.029948
dtype: float64

Top 10 features most negatively correlated with p1_role_DUO:
p3_role_SOLO      -0.430568
p1_role_SOLO      -0.346102
p3_lane_MIDDLE    -0.266160
p1_lane_JUNGLE    -0.184476
p1_role_NONE      -0.184476
p1_role_SUPPORT   -0.101623
p5_role_SUPPORT   -0.076555
p1_lane_TOP       -0.062907
p3_role_CARRY     -0.043606
p4_lane_BOTTOM    -0.041399
dtype: float64


e:\Users\qesterius\Documents\Studia\sztuczna-inteligencja-w-systemach-inf\.venv\lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
e:\Users\qesterius\Documents\Studia\sztuczna-inteligencja-w-systemach-inf\.venv\lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


NaN correlations = 85

Top 10 features most positively correlated with p1_role_SOLO:
p1_lane_TOP        0.863879
p3_lane_MIDDLE     0.377829
p3_role_SOLO       0.351761
p2_lane_JUNGLE     0.328411
p2_role_NONE       0.328411
p7_role_NONE       0.305248
p7_lane_JUNGLE     0.305248
p4_lane_BOTTOM     0.298898
p10_lane_BOTTOM    0.258807
p5_lane_BOTTOM     0.256965
dtype: float64

Top 10 features most negatively correlated with p1_role_SOLO:
p1_lane_JUNGLE    -0.650661
p1_role_NONE      -0.650661
p1_role_SUPPORT   -0.358432
p5_lane_NONE      -0.351867
p7_lane_NONE      -0.351867
p3_lane_NONE      -0.351867
p8_lane_NONE      -0.351867
p9_lane_NONE      -0.351867
p10_lane_NONE     -0.351867
p2_lane_NONE      -0.351867
dtype: float64


e:\Users\qesterius\Documents\Studia\sztuczna-inteligencja-w-systemach-inf\.venv\lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
e:\Users\qesterius\Documents\Studia\sztuczna-inteligencja-w-systemach-inf\.venv\lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


NaN correlations = 85

Top 10 features most positively correlated with p1_role_NONE:
p1_lane_JUNGLE     1.000000
p3_role_SOLO       0.197807
p7_role_NONE       0.169963
p7_lane_JUNGLE     0.169963
p3_lane_MIDDLE     0.150896
p6_lane_TOP        0.150722
p2_lane_JUNGLE     0.146238
p2_role_NONE       0.146238
p10_lane_BOTTOM    0.135494
p9_lane_BOTTOM     0.132091
dtype: float64

Top 10 features most negatively correlated with p1_role_NONE:
p1_lane_TOP       -0.695468
p1_role_SOLO      -0.650661
p1_role_SUPPORT   -0.191049
p2_lane_NONE      -0.187549
p10_lane_NONE     -0.187549
p5_lane_NONE      -0.187549
p9_lane_NONE      -0.187549
p8_lane_NONE      -0.187549
p7_lane_NONE      -0.187549
p6_lane_NONE      -0.187549
dtype: float64


In [19]:
#write collumns to file
with open("data_csv/data_cleaned_columns.txt","w") as f:
    f.write("\n".join(data_cleaned.columns))

In [20]:
models = [
    LogisticRegression(random_state=42, n_jobs=-1),
    DecisionTreeClassifier(random_state=42),
    RandomForestClassifier(random_state=42, n_jobs=-1),
    HistGradientBoostingClassifier(random_state=42),
]

In [21]:
timestamps_starting = data_cleaned[data_cleaned["timeStamp"] == 0]
timestamps_20p = data_cleaned.loc[
    data_cleaned.groupby("matchId")["percentTimeStamp"].apply(
        lambda x: (x - 0.2).abs().idxmin()
    )
]
timestamps_40p = data_cleaned.loc[
    data_cleaned.groupby("matchId")["percentTimeStamp"].apply(
        lambda x: (x - 0.4).abs().idxmin()
    )
]
timestamps_60p = data_cleaned.loc[
    data_cleaned.groupby("matchId")["percentTimeStamp"].apply(
        lambda x: (x - 0.6).abs().idxmin()
    )
]
timestamps_80p = data_cleaned.loc[
    data_cleaned.groupby("matchId")["percentTimeStamp"].apply(
        lambda x: (x - 0.8).abs().idxmin()
    )
]
timestamps_100p = data_cleaned.loc[
    data_cleaned.groupby("matchId")["percentTimeStamp"].apply(
        lambda x: (x - 1.0).abs().idxmin()
    )
]
others = ["matchId"]

timestamps = []
X_0 = timestamps_starting.drop(columns=["first_win"] + others)
y_0 = timestamps_starting["first_win"]
timestamps.append((X_0, y_0, "0"))

X_20 = timestamps_20p.drop(columns=["first_win"] + others)
y_20 = timestamps_20p["first_win"]
timestamps.append((X_20, y_20, "20"))

X_40 = timestamps_40p.drop(columns=["first_win"] + others)
y_40 = timestamps_40p["first_win"]
timestamps.append((X_40, y_40, "40"))

X_60 = timestamps_60p.drop(columns=["first_win"] + others)
y_60 = timestamps_60p["first_win"]
timestamps.append((X_60, y_60, "60"))

X_80 = timestamps_80p.drop(columns=["first_win"] + others)
y_80 = timestamps_80p["first_win"]
timestamps.append((X_80, y_80, "80"))

X_100 = timestamps_100p.drop(columns=["first_win"]+ others)
y_100 = timestamps_100p["first_win"]
timestamps.append((X_100, y_100, "100"))


In [22]:

for ts, (X, Y)  in enumerate([(X_0, y_0), (X_20, y_20), (X_40, y_40), (X_60, y_60), (X_80, y_80), (X_100, y_100)]):
    print("Timestamp: ", ts*20)
    print_correlation_features(X, Y)
    print("\n========================================================\n")
    

Timestamp:  0


e:\Users\qesterius\Documents\Studia\sztuczna-inteligencja-w-systemach-inf\.venv\lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
e:\Users\qesterius\Documents\Studia\sztuczna-inteligencja-w-systemach-inf\.venv\lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


NaN correlations = 470

Top 10 features most positively correlated with first_win:
p9_championId_145    0.023428
p2_championId_245    0.023242
p4_championId_29     0.021595
p6_championId_41     0.021290
p3_lane_TOP          0.020784
p3_role_SUPPORT      0.019769
p5_championId_40     0.019670
p8_summonerLevel     0.019631
p3_summoner2Id_21    0.019511
p1_role_DUO          0.019352
dtype: float64

Top 10 features most negatively correlated with first_win:
p1_role_SOLO        -0.033726
p10_championId_40   -0.030101
p1_lane_TOP         -0.028610
p1_championId_43    -0.021687
p3_lane_MIDDLE      -0.021588
gameDuration_y      -0.021528
gameDuration_x      -0.021528
p9_championId_29    -0.021136
p2_summonerLevel    -0.020640
p2_championId_102   -0.020368
dtype: float64


Timestamp:  20
NaN correlations = 88

Top 10 features most positively correlated with first_win:
blue_currentGold                                  0.077101
p2_currentGold                                    0.059268
p4_totalGo

In [23]:
for i in range(6):
    test_timestamp(timestamps[i])
    print("\n========================================================\n")

Timestamp at 0 percent

   Model: LogisticRegression
Accuracy: 0.5043

   Model: DecisionTreeClassifier
Accuracy: 0.4968

   Model: RandomForestClassifier
Accuracy: 0.5202

   Model: HistGradientBoostingClassifier
Accuracy: 0.5208



Timestamp at 20 percent

   Model: LogisticRegression
Accuracy: 0.5003

   Model: DecisionTreeClassifier
Accuracy: 0.5585

   Model: RandomForestClassifier
Accuracy: 0.6002

   Model: HistGradientBoostingClassifier
Accuracy: 0.6170



Timestamp at 40 percent

   Model: LogisticRegression
Accuracy: 0.5003

   Model: DecisionTreeClassifier
Accuracy: 0.6148

   Model: RandomForestClassifier
Accuracy: 0.6806

   Model: HistGradientBoostingClassifier
Accuracy: 0.6985



Timestamp at 60 percent

   Model: LogisticRegression
Accuracy: 0.5003

   Model: DecisionTreeClassifier
Accuracy: 0.6847

   Model: RandomForestClassifier
Accuracy: 0.7535

   Model: HistGradientBoostingClassifier
Accuracy: 0.7663



Timestamp at 80 percent

   Model: LogisticRegression
Accurac